In [1]:
import pickle

from synbio.annotations import Location, Part

import z3
from z3helpers.definitions import *
from z3helpers.constraints import *
from z3helpers.utils import add_constraints

# phiX174 specific stuffs
with open("phiX174.pkl", "rb") as handle:
    phiX174 = pickle.load(handle)
    
region = Location(1155, 1521, "FWD")
offset = region.start

geneA = Part(seq=phiX174, name="A gene", kind="CDS", location=region)
geneB = phiX174.annotations["B gene"]

# define z3 variables
T = code_dict(triplet_dna_codons, AminoBitVecSort)
dna_seq = dna_variables(region, NucleotideEnumSort)
geneA_prot_seq = protein_variables(geneA, AminoBitVecSort)
geneB_prot_seq = protein_variables(geneB, AminoBitVecSort)

# add translation constraints
geneA_translation_constraints = translation_constraints(
    T, dna_seq, geneA_prot_seq, geneA.location, aminos=z3_bitvec_aminos, offset=offset)
geneB_translation_constraints = translation_constraints(
    T, dna_seq, geneB_prot_seq, geneB.location, 
    start_flag=True, stop_flag=True, aminos=z3_bitvec_aminos, offset=offset)

# add soft constraints
translation_obj = translates_same(geneA_prot_seq, geneA, amino_to_z3_bitvec_amino)
translation_obj += translates_same(geneB_prot_seq, geneB, amino_to_z3_bitvec_amino)

In [12]:
# set up z3
z3.set_param("verbose", 10)
z3.set_param("smt.bv.eq_axioms", False)
z3.set_param("smt.phase_caching_on", 80000)
solver = z3.Optimize()
# add hard constraints
add_constraints(solver, amino_bitvec_unary_restriction())
add_constraints(
    solver, RED20(T, aminos=z3_bitvec_aminos, amino_dict=amino_to_z3_bitvec_amino)
)
add_constraints(solver, geneA_translation_constraints)
add_constraints(solver, geneB_translation_constraints)
# add soft constraints
add_constraints(solver, translation_obj, hard=False)
# solve
if solver.check() == z3.sat:
    print(solver.model())
else:
    print(z3.unsat)

unsat


# Some multiplexing
Goal: probe problem complexity by testing subsets of the problem, i.e., recoding the first $N$ codons of gene B embedded in gene A.

In [2]:
# import pickle
from typing import Optional
import pickle

from synbio.annotations import Location, Part
from synbio.polymers import DNA

import z3
from z3helpers.definitions import *
from z3helpers.constraints import *
from z3helpers.utils import add_constraints

# phiX174 specific stuffs
with open("phiX174.pkl", "rb") as handle:
    phiX174 = pickle.load(handle)
    
NUM_CODONS = 30
A_START = 1155
A_END = 1521
region = Location(A_START, A_END, "FWD")

geneA = Part(seq=phiX174, name="A gene", kind="CDS", location=region)
geneB = phiX174.annotations['B gene']

def get_part(base_part: Part, num_codons: int, start: Optional[int] = None) -> Part:
    base_loc = base_part.location
    base_seq = base_part._seq_reference
    base_name = base_part.name
    base_kind = base_part.kind
    
    if start is None:
        start = base_loc.start
    
    new_loc = Location(start, int(start + 3*num_codons), "FWD")
    return Part(seq=base_seq, name=f"{base_name} ({num_codons} codons)", 
                kind=base_kind, location=new_loc)

subset_A = get_part(geneA, NUM_CODONS, start=A_START)
subset_B = get_part(geneB, NUM_CODONS)

# define z3 variables
T = code_dict(triplet_dna_codons)
dna_seq = dna_variables(region)
geneA_prot_seq = protein_variables(subset_A)
geneB_prot_seq = protein_variables(subset_B)

# add translation constraints
geneA_translation_constraints = translation_constraints(
    T, dna_seq, geneA_prot_seq, subset_A.location, offset=A_START)
geneB_translation_constraints = translation_constraints(
    T, dna_seq, geneB_prot_seq, subset_B.location, 
    start_flag=True, offset=A_START)

# add soft constraints
translation_obj = translates_same(geneA_prot_seq, subset_A)
translation_obj += translates_same(geneB_prot_seq, subset_B)

# set up z3
z3.set_param("verbose", 10)
z3.set_param("smt.bv.eq_axioms", False)
z3.set_param("smt.phase_caching_on", 80000)
solver = z3.Optimize()
# solver.set("timeout", 1000)
# add hard constraints
add_constraints(solver, RED20(T))
add_constraints(solver, geneA_translation_constraints)
add_constraints(solver, geneB_translation_constraints)
# add soft constraints
add_constraints(solver, translation_obj, hard=False)
# solve
solver.check()
m = solver.model()

In [38]:
from z3helpers.utils import rmap
from z3helpers.definitions import dna_to_z3nucleotide as dna_dict
from z3helpers.analysis import objective_function

from synbio.polymers import DNA

def code_from_model(T, model):
    rna = lambda dna: dna.replace("T", "U")
    amino = lambda codon: str(model[T[codon]])
    
    dict_ = {
        rna(codon): amino(codon) for codon in T.keys()
    }
    
    return Code(dict_)

class AmbiguousDNA(DNA):
    basepairing = {
        k:v for k,v in DNA.basepairing.items()
    }
    basepairing['X'] = 'X'
    
    def alphabet(self):
        dna_alphabet = super().alphabet()
        dna_alphabet.append('X')
        return dna_alphabet

def dna_from_model(dna_variables, model):
    dict_ = rmap(dna_dict)
    
    dna_assignments = (
        model[var] for var in dna_seq
    )
    dna_str = (
        'X' if nuc is None else dict_[nuc]
        for nuc in dna_assignments
    )
    return AmbiguousDNA(
        ''.join(dna_str)
    )

    
def prot_from_model(prot_varibles, model):
    amino = lambda z3aa: str(model[z3aa])
    prot_seq = map(amino, prot_varibles)
    
    return Protein(
        ''.join(prot_seq)
    )


# extract values
r20 = code_from_model(T, m)
dna = dna_from_model(dna_seq, m)
pA = prot_from_model(geneA_prot_seq, m)
pB = prot_from_model(geneB_prot_seq, m)

# measure awesomeness
def cost_fxn(out_seq, wt_seq, weights=None):
    if weights is None:
        weights = [1] * len(out_seq)
        
    matches = (
        a1 == a2
        for a1, a2 in zip(out_seq, wt_seq)
    )
    return sum(
        match * weight
        for match, weight in zip(matches, weights)
    )

cost_fxn(pA, subset_A.seq.translate()) + cost_fxn(pB, subset_B.seq.translate())

21